In [1]:
from lwhf.portfolio.backtest import BackTester

2024-06-12 18:53:10.543829: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-12 18:53:10.548697: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-12 18:53:10.608943: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 18:53:11.741282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
bt = BackTester('2024-05-27',3)

In [3]:
bt.get_all_data()
bt.train_model()
bt.backtest()

le-wagon-hedge-fund-data_alpaca_20240604-SP500_Historical_Weekly-2016-01-04-2024-05-27.csv
✅ Found le-wagon-hedge-fund-data_alpaca_20240604-SP500_Historical_Weekly-2016-01-04-2024-05-27.csv in the local cache.
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step


ArpackNoConvergence: ARPACK error -1: ARPACK error -1: No convergence (4211 iterations, 0/1 eigenvectors converged)


        CVXPY note: This failure was encountered while trying to certify
        that a matrix is positive semi-definite (see [1] for a definition).
        In rare cases, this method fails for numerical reasons even when the matrix is
        positive semi-definite. If you know that you're in that situation, you can
        replace the matrix A by cvxpy.psd_wrap(A).

        [1] https://en.wikipedia.org/wiki/Definite_matrix
        

NameError: name 'df' is not defined